In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('combined_scrape.csv')

In [3]:
df.shape

(1808142, 4)

In [4]:
df.head()

,Comment Text,Author,Subreddit,Post ID
0,"Just curious, but what triggered your feelings...",PDXorax,YangForPresidentHQ,ew5ohva
1,Christmas time put up a light-up Yang sign!,PDXorax,YangForPresidentHQ,ew5o8vv
2,"During the general election, get in contact wi...",PDXorax,YangForPresidentHQ,ew42bdx
3,"We need to understand as a country, that the m...",PDXorax,YangForPresidentHQ,evva7l8
4,"Part of the reason for this, is because we nee...",PDXorax,YangForPresidentHQ,evoo179


In [5]:
df.dtypes

Comment Text    object
Author          object
Subreddit       object
Post ID         object
dtype: object

In [6]:
sum_series = df.groupby(['Author','Subreddit']).size()

In [7]:
users = list(set([x for x in sum_series.index.get_level_values(0)]))

In [8]:
len(users)

2882

In [9]:
list_of_subs2 = ['nfl','MMA','CFB','PoliticalHumor','teenagers',
                'soccer','freefolk','gaming','hockey','Libertarian','gameofthrones','MLS',
               ' JoeRogan','conspiracy','marvelstudios','television','asoiaf','leagueoflegends',
                'atheism','samharris','fantasyfootball','CHIBears','hiphopheads',
                'technology','survivor','canada','Futurology','exmormon','Destiny','memes',
                'CryptoCurrency','rupaulsdragrace','Music','dankmemes','stupidpol',
                'Games','FortNiteBR','BigBrother','redsox','baseball','Conservative','Economics',
                'ClashRoyale','DestinyTheGame','europe','NintendoSwitch','IntellectualDarkWeb',
                'askgaybros','hillaryclinton','tuesday','2007scape','tulsi',
                'NASCAR','trees','HistoryMemes','TheSilphRoad','pokemon','Brawlstars','environment','fo76',
                'NYYankees','Android','Overwatch','aznidentity','barstoolsports','wholesomememes',
                'JordanPeterson','TheLastAirbender','golf','DotA2','space','collapse']

In [10]:
features = {}
political_dict={}
for user in users:
    feature_dict = {}
    for sub in sum_series[user].index:
        if sub in list_of_subs2:
            feature_dict[sub]=sum_series[user][sub]
    if len(feature_dict)!=0:
        features[user]=feature_dict
        #create target series
        inner_dict = {}
        try:
            inner_dict['yang'] = sum_series[user]['YangForPresidentHQ']
        except:
            inner_dict['yang'] = 0
        try:
            inner_dict['bernie'] = sum_series[user]['SandersForPresident']
        except:
            inner_dict['bernie'] = 0
        try:
            inner_dict['bernie'] += sum_series[user]['wayofthebern']
        except:
            inner_dict['bernie'] += 0
        try:
            inner_dict['bernie'] += sum_series[user]['Political_Revolution']
        except:
            inner_dict['bernie'] += 0
        try:
            inner_dict['pete'] = sum_series[user]['Pete_Buttigieg']
        except:
            inner_dict['pete'] = 0
        try:
            inner_dict['kamala'] = sum_series[user]['Kamala']
        except:
            inner_dict['kamala'] = 0
        try:
            inner_dict['biden'] = sum_series[user]['JoeBiden']
        except:
            inner_dict['biden'] = 0
        political_dict[user]=inner_dict


In [11]:
df_feature = pd.DataFrame(features).T

In [12]:
df_target = pd.DataFrame(political_dict).T

In [13]:
def func(row):
    total = sum(row)
    if (row['bernie']/total >= .8) & (row['bernie']>=4):
        return 'bernie'
    if (row['biden']/total >= .8) & (row['biden']>=4):
        return 'biden'
    if (row['kamala']/total >= .8) & (row['kamala']>=4):
        return 'kamala'
    if (row['pete']/total >= .8) & (row['pete']>=4):
        return 'pete'
    if (row['yang']/total >= .8) & (row['yang']>=4):
        return 'yang'
    return 'unknown'

In [14]:
df_target['target'] = df_target.apply(func,axis=1)

In [15]:
df_target.drop(['bernie','biden','kamala','pete','yang'],axis=1,inplace=True)

In [16]:
df_target.target.value_counts()

unknown    743
bernie     655
yang       502
pete       312
kamala      73
biden       60
Name: target, dtype: int64

In [17]:
df_feature.head()

,2007scape,Android,BigBrother,Brawlstars,CFB,CHIBears,ClashRoyale,Conservative,CryptoCurrency,Destiny,...,space,stupidpol,survivor,technology,teenagers,television,trees,tuesday,tulsi,wholesomememes
etherpromo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,...,3.0,NaN,NaN,3.0,NaN,NaN,6.0,NaN,NaN,5.0
EndTimesRadio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,...,1.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Riptide360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
Suzina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN
bczeon27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_feature.fillna(0,inplace=True)

In [19]:
#add code to combine like minded subs
df_feature['got']=df_feature['gameofthrones']+df_feature['asoiaf']+df_feature['freefolk']
df_feature['soccer_comb']=df_feature['MLS']+df_feature['soccer']
df_feature['football']=df_feature['nfl']+df_feature['CFB']+df_feature['CHIBears']
df_feature['baseball_comb']=df_feature['baseball']+df_feature['redsox']+df_feature['NYYankees']
df_feature['conservative_comb']=df_feature['Conservative']+df_feature['tuesday']
df_feature['games_comb']=df_feature['gaming']+df_feature['Games']
df_feature['memes_comb']=df_feature['memes']+df_feature['dankmemes']+df_feature['HistoryMemes']+df_feature['wholesomememes']
df_feature['pokemon_comb']=df_feature['TheSilphRoad']+df_feature['pokemon']

In [20]:
#remove above single subs
df_feature.drop(['redsox','baseball','NYYankees','Conservative','tuesday','nfl','CFB','CHIBears','gaming','Games','freefolk','gameofthrones',
                 'asoiaf','memes','dankmemes','HistoryMemes','wholesomememes','TheSilphRoad','pokemon','soccer',
                 'MLS'],axis=1,inplace=True)

In [21]:
df_feature = df_feature.div(df_feature.sum(axis=1), axis=0)

In [22]:
df_feature.head(20)

,2007scape,Android,BigBrother,Brawlstars,ClashRoyale,CryptoCurrency,Destiny,DestinyTheGame,DotA2,Economics,...,trees,tulsi,got,soccer_comb,football,baseball_comb,conservative_comb,games_comb,memes_comb,pokemon_comb
etherpromo,0.000000,0.0,0.000000,0.000000,0.000000,0.067669,0.0,0.0,0.000000,0.000000,...,0.045113,0.000000,0.503759,0.000000,0.000000,0.007519,0.000000,0.105263,0.105263,0.015038
EndTimesRadio,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.128788,0.000000,0.000000,0.000000,0.106061,0.007576,0.075758,0.007576
Riptide360,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Suzina,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086538,0.000000,0.000000
bczeon27,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FLRSH,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.194853,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000
popularis-socialas,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
skanderbeg7,0.000000,0.0,0.000000,0.000000,0.000000,0.440000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
TheCarnalStatist,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.432203,...,0.000000,0.000000,0.000000,0.000000,0.394068,0.000000,0.148305,0.000000,0.000000,0.000000
BOMBZABOMB,0.000000,0.0,0.000000,0.765172,0.092348,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.139842,0.000000


In [23]:
df = pd.concat([df_feature,df_target],axis=1)

In [24]:
df.shape

(2345, 59)

In [25]:
df=df[df['target']!='unknown']

In [26]:
df.shape

(1602, 59)

In [27]:
X = df.drop(['target'],axis=1)

In [28]:
y = df['target']

In [29]:
y.value_counts()

bernie    655
yang      502
pete      312
kamala     73
biden      60
Name: target, dtype: int64

In [30]:
#removing the biden and kamala members since they're counts are really low

In [31]:
df2 = df[(df['target']!='kamala')&(df['target']!='biden')]

In [32]:
df2 = df2[df2.columns.sort_values()]

In [33]:
df2.columns

Index(['2007scape', 'Android', 'BigBrother', 'Brawlstars', 'ClashRoyale',
       'CryptoCurrency', 'Destiny', 'DestinyTheGame', 'DotA2', 'Economics',
       'FortNiteBR', 'Futurology', 'IntellectualDarkWeb', 'JordanPeterson',
       'Libertarian', 'MMA', 'Music', 'NASCAR', 'NintendoSwitch', 'Overwatch',
       'PoliticalHumor', 'TheLastAirbender', 'askgaybros', 'atheism',
       'aznidentity', 'barstoolsports', 'baseball_comb', 'canada', 'collapse',
       'conservative_comb', 'conspiracy', 'environment', 'europe', 'exmormon',
       'fantasyfootball', 'fo76', 'football', 'games_comb', 'golf', 'got',
       'hillaryclinton', 'hiphopheads', 'hockey', 'leagueoflegends',
       'marvelstudios', 'memes_comb', 'pokemon_comb', 'rupaulsdragrace',
       'samharris', 'soccer_comb', 'space', 'stupidpol', 'survivor', 'target',
       'technology', 'teenagers', 'television', 'trees', 'tulsi'],
      dtype='object')

In [34]:
df2.shape

(1469, 59)

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X2 = df2.drop(['target'],axis=1)

In [37]:
X2.columns

Index(['2007scape', 'Android', 'BigBrother', 'Brawlstars', 'ClashRoyale',
       'CryptoCurrency', 'Destiny', 'DestinyTheGame', 'DotA2', 'Economics',
       'FortNiteBR', 'Futurology', 'IntellectualDarkWeb', 'JordanPeterson',
       'Libertarian', 'MMA', 'Music', 'NASCAR', 'NintendoSwitch', 'Overwatch',
       'PoliticalHumor', 'TheLastAirbender', 'askgaybros', 'atheism',
       'aznidentity', 'barstoolsports', 'baseball_comb', 'canada', 'collapse',
       'conservative_comb', 'conspiracy', 'environment', 'europe', 'exmormon',
       'fantasyfootball', 'fo76', 'football', 'games_comb', 'golf', 'got',
       'hillaryclinton', 'hiphopheads', 'hockey', 'leagueoflegends',
       'marvelstudios', 'memes_comb', 'pokemon_comb', 'rupaulsdragrace',
       'samharris', 'soccer_comb', 'space', 'stupidpol', 'survivor',
       'technology', 'teenagers', 'television', 'trees', 'tulsi'],
      dtype='object')

In [38]:
y2 = df2['target']

In [39]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,stratify = y2, random_state=42)

In [40]:
X2_train.head()

,2007scape,Android,BigBrother,Brawlstars,ClashRoyale,CryptoCurrency,Destiny,DestinyTheGame,DotA2,Economics,...,samharris,soccer_comb,space,stupidpol,survivor,technology,teenagers,television,trees,tulsi
GingerPolarBear,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.985859,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
TheMaxClyde,0.0,0.258065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.096774,0.000000,0.000000
Liverman102,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.592105,0.000000,0.019737,0.000000
OctopodesoftheSea,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.357143,0.000000,0.071429,0.000000,0.000000
EAO48,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.064103


In [41]:
X2_test.head()

,2007scape,Android,BigBrother,Brawlstars,ClashRoyale,CryptoCurrency,Destiny,DestinyTheGame,DotA2,Economics,...,samharris,soccer_comb,space,stupidpol,survivor,technology,teenagers,television,trees,tulsi
dru717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
romjpn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.526316,0.0,0.0,0.0,0.0
ocherthulu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.025,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
baha24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
zakrak4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.064000,0.0,0.0,0.0,0.0


In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
lr2 = LogisticRegression(multi_class='multinomial',solver='lbfgs')

In [44]:
lr2.fit(X2_train,y2_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
lr2.score(X2_train,y2_train)

0.5622161671207992

In [46]:
lr2.score(X2_test,y2_test)

0.483695652173913

In [47]:
results = pd.DataFrame(lr2.coef_.T,X2.columns,lr2.classes_)

In [48]:
y2.value_counts(normalize=True)

bernie    0.445882
yang      0.341729
pete      0.212389
Name: target, dtype: float64

In [49]:
results

,bernie,pete,yang
2007scape,-0.214315,-0.413102,0.627417
Android,-0.071675,-0.608833,0.680509
BigBrother,-0.050679,1.005274,-0.954595
Brawlstars,-0.133896,-0.199382,0.333278
ClashRoyale,-0.280315,0.181182,0.099133
CryptoCurrency,-0.289461,-0.729826,1.019288
Destiny,0.176176,0.253965,-0.430141
DestinyTheGame,0.874151,-0.715114,-0.159036
DotA2,-0.559884,-0.230242,0.790126
Economics,-0.032502,0.698590,-0.666088


In [50]:
results[abs(results['yang']>1)]

,bernie,pete,yang
CryptoCurrency,-0.289461,-0.729826,1.019288
Futurology,-0.271122,-1.044636,1.315758
aznidentity,-1.175711,-0.645512,1.821223
samharris,-1.039766,0.010985,1.028782
